In [6]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import pandas as pd

In [7]:

# Load the CSV into a DataFrame
df = pd.read_csv('../data/prep_dataset.csv')
df.head()


,title,text,tags
0,Introduction to music information retrieval wi...,"In this article, I am going to illustrate some...",Music
1,How Twitter does Automated Testing,It was 8:00 AM and I entered the Zoom call. Ka...,"Web Development, Software Development"
2,Why you should start using short URLs in your ...,Photo by Luke Chesser on Unsplash\r\n\r\nWhen ...,Programming
3,Why music isn’t a top-two category on Patreon ...,"Cherie Hu: Hey, Wyatt, thanks so much for join...","Business, Startup, Music, Technology"
4,Love Not Fear: Freedom of Speech,I saw an article about a popular video platfor...,Love


In [8]:
df['tags'] = df['tags'].apply(lambda x: x.split(','))
print(df['tags'].head())

0                                      [Music]
1     [Web Development,  Software Development]
2                                [Programming]
3    [Business,  Startup,  Music,  Technology]
4                                       [Love]
Name: tags, dtype: object


In [9]:

def recommend_tags(user_tags, df, num_recommendations=10, min_support=0.001):
    # Step 1: Prepare the dataset
    # Convert articles' tags to a one-hot encoded dataframe for apriori algorithm
    transactions = df['tags'].tolist()
    te = TransactionEncoder()
    te_ary = te.fit_transform(transactions)
    transaction_df = pd.DataFrame(te_ary, columns=te.columns_)
    
    # Step 2: Apply Apriori algorithm to find frequent itemsets
    frequent_itemsets = apriori(transaction_df, min_support=min_support, use_colnames=True)    
    
    # Step 3: Generate association rules from frequent itemsets
    num_itemsets = frequent_itemsets.shape[0]
    rules = association_rules(frequent_itemsets, num_itemsets=num_itemsets, metric="support", min_threshold=min_support)

    # Step 4: Filter rules where antecedents overlap with user's tags
    relevant_rules = rules[rules['antecedents'].apply(lambda x: len(x.intersection(user_tags)) > 0)]
    
    # Step 5: Get unique consequents (recommended tags), sorted by support
    recommendations = relevant_rules[['consequents', 'support']].sort_values(by='support', ascending=False)
    recommended_tags = set()
    
    for _, row in recommendations.iterrows():
        recommended_tags.update(row['consequents'])
        if len(recommended_tags) >= num_recommendations:
            break
    
    # Step 6: Include user tags in the recommended tags
    recommended_tags.update(user_tags)
    
    # Step 7: Return top recommendations (limited to num_recommendations)
    return list(recommended_tags)[:num_recommendations]


In [10]:
# Example: User selects some tags
user_tags = {'AI', 'Programming'}
recommended_tags = recommend_tags(user_tags,df)
print("\nUser's Chosen Tags:", user_tags)
print("Recommended Tags:", recommended_tags)


User's Chosen Tags: {'Programming', 'AI'}
Recommended Tags: [' JavaScript', ' Python', ' Startup', 'Programming', ' Machine Learning', 'AI', ' Data Science', ' Web Development', ' Software Development']
